In [1]:
#Web Traffic Prediction
#Temporal Signals Static Graph
import torch
from torch_geometric_temporal.dataset import WikiMathsDatasetLoader
from torch_geometric_temporal.signal import temporal_signal_split

/Users/smudge/Desktop/Code/temporal-graph-seminar/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
loader=WikiMathsDatasetLoader()
dataset=loader.get_dataset(lags=8)
train,test=temporal_signal_split(dataset,train_ratio=0.5)

In [3]:
#Model
from torch_geometric_temporal.nn.recurrent import GConvGRU
from torch.nn import Linear
import torch.nn.functional as F 
from torch.nn import Module

#Gated Recurrent Unit
class TemporalNetwork(Module):
    def __init__(self,features,filters):
        super(TemporalNetwork,self).__init__()

        self.conv_gru=GConvGRU(in_channels=features,out_channels=filters,K=3,normalization='sym')
        self.linear=Linear(in_features=filters,out_features=1)
    
    def forward(self,x,edge_index,edge_weight):
        x=self.conv_gru(x,edge_index,edge_weight).relu()
        x=self.linear(x)

        return x


In [4]:
#call
from torch.optim import Adam
from torch.nn import MSELoss
from tqdm import tqdm

model=TemporalNetwork(features=8,filters=32)
optimizer=torch.optim.Adam(params=model.parameters(),lr=0.001,betas=(0.9,0.999))
loss=MSELoss()

In [5]:
#Training
def training_loop():
    model.train(True)

    NUM_EPOCHS=50

    for epoch in tqdm(range(NUM_EPOCHS)):
        train_loss=0
        test_loss=0
        #Train
        for t,snap in enumerate(train):
            op=model(snap.x,snap.edge_index,snap.edge_weight)
            snap_loss=loss(snap.y,op)

            #training
            model.zero_grad()

            snap_loss.backward()
            optimizer.step()

            #aggregate
            train_loss+=snap_loss.item()

        #Validation
        model.eval()
        
        with torch.no_grad():
            for t,snap in enumerate(test):
                op=model(snap.x,snap.edge_index,snap.edge_weight)
                snap_loss=loss(snap.y,op)

                test_loss+=snap_loss.item()
        
            print("Epoch: {ep}".format(ep=epoch+1))
            print("Train Loss: {l}".format(l=train_loss/(t+1)))
            print("Validation Loss: {l}".format(l=test_loss/(t+1)))






In [6]:
training_loop()

  0%|          | 0/50 [00:00<?, ?it/s]/Users/smudge/Desktop/Code/temporal-graph-seminar/.venv/lib/python3.9/site-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([1068, 1])) that is different to the input size (torch.Size([1068])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
  2%|▏         | 1/50 [00:15<12:28, 15.28s/it]

Epoch: 1
Train Loss: 0.8211720646744933
Validation Loss: 0.7429497517768849


: 